<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
پیش‌بینی زمان سلامت
</font>
</h1>

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معرفی مجموعه داده
</font>
</h2>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>

</font>
</p>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
در این قسمت کتاب‌خانه‌ها و ابزار مورد نیاز خود را 
<code>import</code>
 کنید 
 و
  فایل داده‌ها را که در پوشه‌ی 
  <code>Data</code>
ذخیره‌شده‌اند را بخوانید و وارد محیط کار خود کنید.
</font>
</p>

In [6]:
import pandas as pd
import os

def load_and_combine_data(data_path):
    all_files = [f for f in os.listdir(data_path) if f.endswith('.csv')]
    df_list = []
    for file in all_files:
        df = pd.read_csv(os.path.join(data_path, file))
        # Extracting sensors' names
        sensor_name = file.split('_')[0]
        df.rename(columns={'value': sensor_name}, inplace=True)
        df_list.append(df)

    combined_df = pd.concat(df_list, axis=1)
    if 'RUL_seconds' in combined_df.columns:
        rul_seconds = combined_df['RUL_seconds'].iloc[:, 0]
        combined_df = combined_df.loc[:,~combined_df.columns.duplicated()]
        combined_df['RUL_seconds'] = rul_seconds
    return combined_df

train_path = './train'
test_path = './test'

train_df = load_and_combine_data(train_path)
test_df = load_and_combine_data(test_path)

print("Shape of the training data:", train_df.shape)
print("Shape of the test data:", test_df.shape)

Shape of the training data: (35000, 16)
Shape of the test data: (15000, 15)


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
پیش‌پردازش و مهندسی ویژگی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
        در این سوال شما می‌توانید از هر تکنیک پیش‌پردازش/مهندسی ویژگی دلخواهتان، استفاده کنید.
    <br>
    تکنیک‌هایی که استفاده می‌کنید به شکل مستقیم مورد ارزیابی توسط سامانه داوری قرار <b>نمی‌گیرند.</b> بلکه همه آن‌ها در دقت مدل شما تاثیر خواهند گذاشت؛ بنابراین هر چه پیش‌پردازش/مهندسی ویژگی بهتری انجام دهید تا دقت مدل بهبود پیدا کند، امتیاز بیشتری از این سوال کسب خواهید کرد.
    در این قسمت شما می‌توانید بخشی از داده‌ی موجود را برای اعتبارسنجی در نظر بگیرید.
</font>
</p>

In [7]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

X_train = train_df.drop('RUL_seconds', axis=1)
y_train = train_df['RUL_seconds']
X_test = test_df

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_part, X_val, y_train_part, y_val = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
آموزش مدل
</font>
</h2>



In [8]:
# Creating LGMB model
lgbm = lgb.LGBMRegressor(random_state=42)
lgbm.fit(X_train_part, y_train_part)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000686 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3825
[LightGBM] [Info] Number of data points in the train set: 28000, number of used features: 15
[LightGBM] [Info] Start training from score 452.496714


LGBMRegressor(random_state=42)

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معیار ارزیابی
</font>
</h2>



In [9]:
# Evaluating using validation
y_pred_val = lgbm.predict(X_val)
r2 = r2_score(y_val, y_pred_val)
print(f"R2 Score on validation set with LightGBM: {r2}")

# Training on all the train data
lgbm.fit(X_train_scaled, y_train)

from sklearn.model_selection import GridSearchCV

# Search space for finding Hyper Parameters
param_grid = {
    'n_estimators': [100, 200, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [31, 50, 100],
}

# Creating an object of GridSearchCV
grid_search = GridSearchCV(estimator=lgb.LGBMRegressor(random_state=42),
                           param_grid=param_grid,
                           cv=3,
                           scoring='r2',
                           n_jobs=-1,
                           verbose=2)

# Network searching
grid_search.fit(X_train_scaled, y_train)

print("Best parameters found: ", grid_search.best_params_)

best_model = grid_search.best_estimator_

R2 Score on validation set with LightGBM: 0.7716648639685396
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000831 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3825
[LightGBM] [Info] Number of data points in the train set: 35000, number of used features: 15
[LightGBM] [Info] Start training from score 452.436771
Fitting 3 folds for each of 27 candidates, totalling 81 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000777 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3825
[LightGBM] [Info] Number of data points in the train set: 35000, number of used features: 15
[LightGBM] [Info] Start training from score 452.436771
Best parameters found:  {'learning_rate': 0.1, 'n_estimators': 100, 'num_leaves': 31}


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
 پیش‌بینی برای داده تست و خروجی
</font>
</h2>


In [10]:
predictions = best_model.predict(X_test_scaled)

submission_df = pd.DataFrame({'prediction': predictions})

submission_df.to_csv('submission.csv', index=False)

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
<b>سلول جواب‌ساز</b>
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    برای ساخته‌شدن فایل <code>result.zip</code> سلول زیر را اجرا کنید. توجه داشته باشید که پیش از اجرای سلول زیر تغییرات اعمال شده در نت‌بوک را ذخیره کرده باشید (<code>ctrl+s</code>) در غیر این صورت، در پایان مسابقه نمره شما به صفر تغییر خواهد کرد.
    <br>
    همچنین اگر از کولب برای اجرای این فایل نوت‌بوک استفاده می‌کنید، قبل از ارسال فایل <code>result.zip</code>، آخرین نسخه‌ی نوت‌بوک خود را دانلود کرده و داخل فایل ارسالی قرار دهید.
</font>

In [11]:
import zipfile
import joblib
import os

if not os.path.exists(os.path.join(os.getcwd(), 'notebook.ipynb')):
    %notebook -e notebook.ipynb

def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)

submission_df.to_csv('submission.csv', index=False)
file_names = ['notebook.ipynb', 'submission.csv']
compress(file_names)

File Paths:
['notebook.ipynb', 'submission.csv']
